# Global Drifter Program (GDP)

As part of this Notebook, we will use the *GDP historical dataset* to highlight the required steps to preprocess and dataset into a format that can be ingest by the *CloudDrift* library.

## Dataformat module

The `dataformat.py` module contains the class `create_ragged_array` to transform a series of archives into an *Awkward Array* where all variables is stored as a *ragged array*. The module also contains `read_from_netcdf` and `read_from_parquet` to initialize the *Awkward Array* directly from an previously preprocessed archive. Right now, it *only* supports local array but we will soon add the possibility of *lazy-loading* array stored in the Cloud.

In [1]:
import sys
sys.path.insert(0, '../')
from clouddrift.dataformat import create_ragged_array, read_from_netcdf, read_from_parquet

The main class of this module is *create_ragged_array* and is used to create a single archive that can be saved to a netCDF or Parquet file. The signature of the class is:

In [2]:
create_ragged_array?

Init signature:
create_ragged_array(
    indices: list,
    vars_coords: dict,
    vars_meta: list,
    vars_data: list,
    preprocess_func: collections.abc.Callable[[int], xarray.core.dataset.Dataset],
    rowsize_func: collections.abc.Callable[[int], int] = None,
)
Docstring:      <no docstring>
File:           ~/Library/CloudStorage/OneDrive-FloridaStateUniversity/projects/clouddrift/clouddrift/dataformat.py
Type:           type
Subclasses:     


## Dataset-specific functions

Since each dataset is different, we have to create specific functions to preprocess the dataset (`preprocess_func`) and return the metadata and data of a single trajectory. This was inspired by the [Pangeo Forge](https://pangeo-forge.readthedocs.io/en/latest/) project. The class *create_ragged_array* will use those functions to create the single archive of ragged arrays. More precisely, it requires:
- a list of indices (or identification number) that will be concatenate into the ragged array format
- a dictionnary mapping the mandatory coordinates list to the name of those variables in the dataset, e.g.
    coords = {'ids': 'number', 'time': 't', 'longitude': 'lon', 'latitude': 'lat'}
- a list of variable names containing metadata information about the trajectory (size: 1 per trajectory)
- a list of variable names containing the data along the trajectory (size: number of observations per trajectory)
- a preprecessing function with the following signature:
    - `Signature: preprocess_func(index: int) -> xarray.core.dataset.Dataset`, where the index parameter is an identifier of a trajectory, e.g. the identification number of an Argo float) and returns an *xarray Dataset*. 
    
This function can performs all type of operations, such as formatting the date, changing the type of variables, modifying the metadata, etc. We provide preprocessing function for different datasets in the `data/recipes/` folder. The class also needs to *initially* calculate the sum of all observations. By default, this is performed using `lambda i: self.preprocess_func(i).dims['obs']`. To *speed up* this process, in the situation where a lot of preprocessing are performed, it is possible to provide a second function `rowsize_func`, that returns directly the number of observation of a trajectory (`Signature: rowsize_func(index: int) -> int`)

Finally, we included an function to download (or update) the GDP dataset.

In [3]:
%load_ext autoreload
%autoreload 2
from data.gdp6h import preprocess, rowsize, download

Listing AOML directory...
https://www.aoml.noaa.gov/ftp/pub/phod/lumpkin/netcdf/buoydata_1_5000
https://www.aoml.noaa.gov/ftp/pub/phod/lumpkin/netcdf/buoydata_5001_10000
https://www.aoml.noaa.gov/ftp/pub/phod/lumpkin/netcdf/buoydata_10001_15000
https://www.aoml.noaa.gov/ftp/pub/phod/lumpkin/netcdf/buoydata_15001_dec20


# Download

The download function will store the raw dataset into the `data/raw/` folder specified in the `gdp.py` module. By default `download_gdp_data()` will download the complete GPD dataset (containing 17,324 files as of May 2022) from the AOML `https` server.

**Note**: this Notebook is very similar to the `data-glad.ipynb` Notebook because very few functions have to be created to transform a new dataset. We hope that this will encourage people to use this dataformat and utilize the CloudDrift library.

In [4]:
download?

Signature: download(drifter_ids: list = None, n_random_id: int = None)
Docstring:
Download individual netCDF files from the AOML server

:param drifter_ids [list]: list of drifter to retrieve (Default: all)
:param n_random_id [int]: randomly select n drifter netCDF files
:return drifters_ids [list]: list of retrived drifter
File:      ~/Library/CloudStorage/OneDrive-FloridaStateUniversity/projects/clouddrift/data/gdp6h.py
Type:      function


It is possible to prodive a list of `drifter_ids` to retrieve a subset and/or specified a integer `n_random_id` to randomly retrieve `n` trajectory. The function returns the list of `drifters_ids` that was downloaded, and can be passed to create the ragged array.

In [5]:
drifter_ids = download()

100%|██████████████████████████████████████████████████████████████████████████████████████████| 25587/25587 [00:00<00:00, 181701.55it/s]


Once the data downloaded, it is possible to create the ragged array and either save a netCDF, parquet file, or simply output an Awkward Array that can be used for analysis.

In [6]:
drifter_ids

array([ 7700130,  7700131,  7700132, ..., 68764470, 68766460, 68768470])

In [7]:
coords = {'ids': 'ids', 'time': 'time', 'lon': 'longitude', 'lat': 'latitude'}
metadata = ['ID', 'rowsize', 'WMO', 'expno', 'deploy_date', 'deploy_lat', 'deploy_lon', 'end_date', 'end_lat', 'end_lon', 'drogue_lost_date', 'typedeath', 'typebuoy', 'DeployingShip', 'DeploymentStatus', 'BuoyTypeManufacturer', 'BuoyTypeSensorArray', 'CurrentProgram', 'PurchaserFunding', 'SensorUpgrade', 'Transmissions', 'DeployingCountry', 'DeploymentComments', 'ManufactureYear', 'ManufactureMonth', 'ManufactureSensorType', 'ManufactureVoltage', 'FloatDiameter', 'SubsfcFloatPresence', 'DrogueType', 'DrogueLength', 'DrogueBallast', 'DragAreaAboveDrogue', 'DragAreaOfDrogue', 'DragAreaRatio', 'DrogueCenterDepth', 'DrogueDetectSensor']
data = ['ve', 'vn', 'temp', 'err_lat', 'err_lon', 'err_temp', 'drogue_status']

ra = create_ragged_array(drifter_ids,
                         coords, 
                         metadata, 
                         data,
                         preprocess_func=preprocess,
                         rowsize_func=rowsize
                        )

Calculating the number of observations: 100%|██████████████████████████████████████████████████████| 25587/25587 [14:34<00:00, 29.28it/s]
/Users/pmiron/miniconda3/envs/clouddrift/lib/python3.10/site-packages/xarray/coding/variables.py:140: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv
Filling the ragged array: 100%|████████████████████████████████████████████████████████████████████| 25587/25587 [27:41<00:00, 15.40it/s]


## Export

In [8]:
ra.to_parquet('../data/process/gdp_6h_dec20.parquet')
ra.to_netcdf('../data/process/gdp_6h_dec20.nc')

In [119]:
ds = ra.to_awkward()

In [120]:
ds.ID

<Array [7700130, 7700131, ..., 68768470] type='25587 * int64[parameters={"a...'>

In [121]:
ds.fields

['ID',
 'rowsize',
 'WMO',
 'expno',
 'deploy_date',
 'deploy_lat',
 'deploy_lon',
 'end_date',
 'end_lat',
 'end_lon',
 'drogue_lost_date',
 'typedeath',
 'typebuoy',
 'DeployingShip',
 'DeploymentStatus',
 'BuoyTypeManufacturer',
 'BuoyTypeSensorArray',
 'CurrentProgram',
 'PurchaserFunding',
 'SensorUpgrade',
 'Transmissions',
 'DeployingCountry',
 'DeploymentComments',
 'ManufactureYear',
 'ManufactureMonth',
 'ManufactureSensorType',
 'ManufactureVoltage',
 'FloatDiameter',
 'SubsfcFloatPresence',
 'DrogueType',
 'DrogueLength',
 'DrogueBallast',
 'DragAreaAboveDrogue',
 'DragAreaOfDrogue',
 'DragAreaRatio',
 'DrogueCenterDepth',
 'DrogueDetectSensor',
 'obs']

In [122]:
ds.obs.fields

['ids',
 'time',
 'longitude',
 'latitude',
 've',
 'vn',
 'temp',
 'err_lat',
 'err_lon',
 'err_temp',
 'drogue_status']

## Read

In [20]:
import awkward._v2 as ak

In [12]:
ds2 = read_from_parquet('../data/process/gdp_6h_dec20.parquet')

In [13]:
ds2.ID

<Array [7700130, 7700131, ..., 68768470] type='25587 * int64[parameters={"a...'>